In [1]:
# Membantu mem-fetch halaman
from selectolax.parser import HTMLParser

# Membantu mengolah data
import pandas as pd
import numpy as np
from glob import glob
from ast import literal_eval
from datetime import timedelta

from tqdm import tqdm

import plotly.express as px
import plotly.graph_objects as go

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
def get_data(filename: str):
  ''' Ekstrak data gempa dari filename'''

  with open(filename) as f:
    # baca berkas HTML
    text = f.read()

  # Ambil data gempa. Data terletak di tag <script> ketiga dari terakhir.
  the_script = HTMLParser(text).css('script')[-3].text()
  events = the_script.split('var locations =')[1].split(';\n')[0].strip()
  sensors = the_script.split('var sensors =')[1].split(';\n')[0].strip()

  events = literal_eval(events) if events!='null' else []
  sensors = literal_eval(sensors) if sensors!='null' else []
  
  return events, sensors

In [3]:
events = []
sensors = []

# Semua berkas HTML yang sudah kita unduh berada di dua folder ini
filelist = glob('./output/*.html')
filelist.extend(glob('./output_recheck/*.html'))


for filename in tqdm(filelist):
    eves, sens = get_data(filename)
    events.extend(eves)
    sensors.extend(sens)

100%|██████████| 1072/1072 [00:33<00:00, 32.00it/s]


In [4]:
# Daftar kolom di data Gempa
columns = [
    'No', 'eventID', 'datetime', 'latitude', 'longitude', 'magnitude', 'mag_type',
    'depth', 'phasecount', 'azimuth_gap', 'location', 'agency',  'datetimeFM',
    'latFM', 'lonFM', 'magFM', 'magTypeFM', 'depthFM', 'phasecountFM', 'AzGapFM',
    'scalarMoment', 'Mrr', 'Mtt', 'Mpp', 'Mrt', 'Mrp', 'Mtp', 'varianceReduction',
    'doubleCouple', 'clvd', 'strikeNP1', 'dipNP1', 'rakeNP1', 'strikeNP2', 'dipNP2',
    'rakeNP2', 'azgapFM', 'misfit',
]

# buat DataFrame
df = pd.DataFrame(events, columns=columns)

# Beberapa perapian kecil:
# ubah kolom datetime dari string ke objek datetime
df.datetime = pd.to_datetime(df.datetime)
# buang kolom 'No' (Nomor)
df = df.drop(columns=['No'])
# buang data duplikat, jika ada
df = df.drop_duplicates()
# urutkan data berdasarkan datetime.
df = df.sort_values(by='datetime').reset_index(drop=True)

In [5]:
len(df), len(set(df.eventID))

(113087, 113087)

In [6]:
# cek selisih waktu event gempa dengan event sebelumnya
time_diff = df.datetime.sort_values().diff(periods=1)

# tampilkan semua selisih hari antar event gempa
set(time_diff.apply(lambda x: x.days))
# {0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, nan, 27.0}

{0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, nan, 27.0}

In [7]:
df1 = df.copy()

calendar = df1\
    .set_index('datetime').resample('D')['magnitude'].count()\
    .resample('30D').apply(list).to_list()[:-1]

fig = px.imshow(np.array(calendar).T, zmax=1)
fig.show()

In [8]:
# dfold = pd.read_csv('earthquakes.tsv', sep='\t')
# dfold.datetime = pd.to_datetime(dfold.datetime, format='ISO8601')

# timeold = set(dfold.datetime.apply(lambda x:x.date()))
# timenew = set(df.datetime.apply(lambda x:x.date()))
# timeold - timenew

In [9]:
df.to_csv('katalog_gempa_v2.tsv', sep='\t')

### To Check

Ada beberapa tanggal di grafik pada Bagian tadi, yang tidak memiliki catatan data gempa (ditandai oleh warna biru). Hal ini dapat berarti memang tidak ada satupun gempa di hari itu (yang mungkin dihasilkan dari basis data yang rusak, suatu perawatan tingkat nasional yang mengakibatkan tidak ada sensor beroperasi pada hari itu, dll.), atau yang lebih mungkin: **kita gagal mengunduh data gempa hari itu**.

Untuk memastikan, kita akan mengumpulkan semua tanggal dengan data yang hilang, lalu mengunduhnya satu-per-satu (delta waktu 1 hari).

In [10]:
timeline = df.datetime

aday = timedelta(1)
to_check = set()

for x, y in zip(timeline, timeline[1:]):
    if (y-x).days < 1: continue
    # if x.year < 2017: continue
    start, end = x.date(), y.date()
    while start<end:
        to_check.add(start)
        start += aday

len(to_check)

155

In [11]:
import pickle

with open('to_check.pickle', 'wb') as f:
    pickle.dump(sorted(to_check, reverse=True), f)

In [12]:
to_check

{datetime.date(2008, 11, 11),
 datetime.date(2008, 11, 12),
 datetime.date(2008, 11, 15),
 datetime.date(2008, 11, 16),
 datetime.date(2008, 11, 17),
 datetime.date(2008, 12, 2),
 datetime.date(2008, 12, 9),
 datetime.date(2008, 12, 10),
 datetime.date(2008, 12, 11),
 datetime.date(2008, 12, 12),
 datetime.date(2008, 12, 13),
 datetime.date(2008, 12, 14),
 datetime.date(2008, 12, 27),
 datetime.date(2009, 1, 9),
 datetime.date(2009, 1, 28),
 datetime.date(2009, 1, 29),
 datetime.date(2009, 2, 4),
 datetime.date(2009, 2, 5),
 datetime.date(2009, 5, 11),
 datetime.date(2009, 6, 2),
 datetime.date(2009, 7, 3),
 datetime.date(2009, 7, 4),
 datetime.date(2009, 7, 5),
 datetime.date(2009, 7, 6),
 datetime.date(2009, 7, 24),
 datetime.date(2009, 8, 23),
 datetime.date(2009, 8, 24),
 datetime.date(2009, 8, 25),
 datetime.date(2009, 8, 26),
 datetime.date(2009, 8, 27),
 datetime.date(2009, 8, 28),
 datetime.date(2009, 8, 29),
 datetime.date(2009, 8, 30),
 datetime.date(2009, 8, 31),
 datetime.d

## Sensors

In [13]:
df = pd.DataFrame(sensors, columns='col1 code latitude longitude elevation location datetime col2'.split())

# Hanya ada IA
df = df[df.col1=='IA']
df = df.drop(columns=['col1', 'col2'])
df = df.drop_duplicates()


In [14]:
df.to_csv('katalog_sensor.tsv', sep='\t')